# Задачи Нужно реализовать простейшую семантическую поисковую систему помощью векторного представления предложений/текстов.

## 1. Загрузка и предварительная обработка данных

In [1]:
import gzip
import pandas as pd
import re
import spacy
import pickle  # 用于保存和加载DataFrame

# 加载并解压数据 (Load and decompress data)
def load_data_from_gz(gz_path):
    with gzip.open(gz_path, 'rt', encoding='utf-8') as gz_file:
        file_content = gz_file.read().strip()
    data = [line.split('\t') for line in file_content.splitlines() if len(line.split('\t')) == 3]
    # 将文件内容按行分割，并确保每行有三个部分（类别、标题、内容）
    # Split the file content by lines and ensure each line has three parts (category, title, content)
    df = pd.DataFrame(data, columns=['category', 'title', 'content'])
    return df

df = load_data_from_gz(r"D:\MyProject\SPBU Course\NLP\nlp_task_3\news.txt.gz")

# 加载spaCy的俄语模型 (Load the Russian model of spaCy)
nlp = spacy.load("ru_core_news_sm")

# 预处理函数：词形还原和去除停用词 (Preprocessing function: lemmatization and stopword removal)
def preprocess(text, stopwords=None):
    text_cleaned = re.sub(r'[^а-яА-Я]', ' ', text.lower())
    doc = nlp(text_cleaned)
    lemmatized_words = [
        token.lemma_ for token in doc 
        if not token.is_stop and not token.is_punct
    ]
    if stopwords:
        lemmatized_words = [word for word in lemmatized_words if word not in stopwords]
    return lemmatized_words

# 定义俄语停用列表 (Define a set of Russian stopwords)
russian_stopwords = set(['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она',
                         'так', 'его', 'но', 'для', 'около', 'же', 'теперь', 'быть', 'бывать', 'этот', 'вот',
                         'чем', 'еще', 'мочь', 'тот', 'когда', 'другой', 'первыи', 'ж', 'там', 'себя'])

df['preprocessed_content'] = df['content'].apply(lambda x: preprocess(x, russian_stopwords))

# 保存预处理后的DataFrame到本地
with open('preprocessed_df.pkl', 'wb') as f:
    pickle.dump(df, f)

# 查看结果
print(df[['content', 'preprocessed_content']].head())

                                             content  \
0  Парусная гонка Giraglia Rolex Cup пройдет в Ср...   
1  Шведский хоккеист Матс Сундин назначен советни...   
2  Гран-при конкурса "Брэнд года/EFFIE" получил г...   
3  Цена американской нефти WTI на лондонской бирж...   
4  Сбербанк выставил на продажу долги по 21,4 тыс...   

                                preprocessed_content  
0  [парусный, гонка,                    , пройти,...  
1  [шведский, хоккеист, матс, сундин, назначить, ...  
2  [гран, конкурса,  , брэнд, год,        , получ...  
3  [цена, американский, нефть,     , лондонский, ...  
4  [сбербанк, выставить, продажа, долг,      , ты...  


## 2. Выберите модель SentenceTransformer
#### В частности, использовалась предварительно обученная модель 'paraphrase-MiniLM-L6-v2'. Эта модель способна преобразовывать текстовые предложения в высокоразмерные векторы (эмбеддинги), которые могут быть использованы для вычисления сходства между предложениями.
## 3. Выбор FAISS в качестве хранилища векторов
#### FAISS - это эффективная библиотека, предназначенная для поиска сходства в больших векторных коллекциях. Она поддерживает широкий спектр типов индексов и работает как на GPU, так и на CPU, обеспечивая быстрый поиск ближайших соседей.

In [2]:
import os
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import pickle

# 定义路径
df_path = 'preprocessed_df.pkl'
embeddings_path = 'all_embeddings.npy'
index_path = 'faiss_index.index'

def create_and_save_data():
    # 加载预处理后的DataFrame (Load the preprocessed DataFrame)
    with open(df_path, 'rb') as f:
        df = pickle.load(f)

    # 加载SentenceTransformer模型 (Load the SentenceTransformer model)
    embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

    # 创建整个数据集的句子嵌入（如果不存在）(Create sentence embeddings for the entire dataset if they do not exist)
    if not os.path.exists(embeddings_path):
        print("Creating embeddings...")
        # 将列表中的单词拼接成句子 (Join words in the list into sentences)
        all_embeddings = embedder.encode(df['preprocessed_content'].apply(lambda x: ' '.join(x)).tolist(), convert_to_tensor=True).cpu().numpy()
        np.save(embeddings_path, all_embeddings)
        print("Embeddings created and saved.")
    else:
        all_embeddings = np.load(embeddings_path)

    # 初始化FAISS索引（如果不存在）(Initialize the FAISS index if it does not exist)
    if not os.path.exists(index_path):
        print("Initializing FAISS index...")
        dimension = all_embeddings.shape[1]  # 获取嵌入维度 (Get the embedding dimension)
        index = faiss.IndexFlatL2(dimension)  # 使用L2距离的平面索引 (Use a flat index with L2 distance)
        index.add(all_embeddings)
        faiss.write_index(index, index_path)
        print("FAISS index initialized and saved.")

# 执行创建和保存数据 (Execute the creation and saving of data)
create_and_save_data()


Creating embeddings...
Embeddings created and saved.
Initializing FAISS index...
FAISS index initialized and saved.
